In [1]:
import numpy as np
from numpy import loadtxt
import pandas as pd
import csv
import math
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Importing training data as X_train and Y_train
file = open('dataset.csv')
csvreader = csv.reader(file, delimiter = ",")
data = []
for row in csvreader:
    data.append(row)

train_data = []
for i in range(0 , len(data)):
    col= []
    for j in range(0,len(data[i])):
        if j==len(data[i])-1:
            col.append(data[i][j])
        else:
            col.append(data[i][j])
    train_data.append(col)
file.close()
X_train = []
Y_train = []
for i in range(len(train_data)):
    cols = []
    for j in range(0,len(train_data[i])):
        if(j==len(train_data[i])-1):
            Y_train.append(train_data[i][j])
        else:
            cols.append(train_data[i][j])
    X_train.append(cols)   
for i in range(0,len(X_train)):
    for j in range(0,len(X_train[i])):
        X_train[i][j]=float(X_train[i][j])
X_train = np.array(X_train)
Y_train = np.array(Y_train)

X_train_df = pd.DataFrame(X_train, columns = ['Height','Weight','Diameter','Color'])
Y_train_df = pd.DataFrame(Y_train,columns = ['Material'])

In [3]:
#Taking out test dataset as X_test and Y_test
test_data = []
c = 0
p = 0
m = 0
k = 0
x = []
for i in range(0,len(train_data)):
    if(train_data[i][4]==' Ceramic' and c<6):
        x.append(i)
        c=c+1
        test_data.append(train_data[i])
    if(train_data[i][4]==' Plastic' and p<6):
        x.append(i)
        p=p+1
        test_data.append(train_data[i])
    if(train_data[i][4]==' Metal' and m<6):
        x.append(i)
        m=m+1
        test_data.append(train_data[i])
    if(p>=6 and c>=6 and m>=6):
            train_data.append(train_data[i])
test_df = pd.DataFrame(data = test_data)
X_test = test_df.drop([4],axis = 'columns')
Y_test = test_df[4]
X_test = np.array(X_test).astype(np.float)
Y_test = np.array(Y_test)

In [4]:
#Deleting the Test Data set form the training set
X_train_df= X_train_df.drop(X_train_df.index[x])
Y_train_df = Y_train_df.drop(Y_train_df.index[x])


In [5]:
X_train = np.array(X_train_df).astype(np.float)
Y_train  = np.array(Y_train_df)
def set_labels_int(Y):
    for i in range(0,len(Y)):
        if Y[i] ==" Metal":
            Y[i]=0
        if Y[i] == " Plastic":
            Y[i]=1
        if Y[i] == " Ceramic":
            Y[i]=2
    return Y
Y_train = set_labels_int(Y_train)
Y_test = set_labels_int(Y_test)

In [6]:
def intialize_centroid(K):
    x = np.random.randint(len(X_train)-1,size=K)
    cent = []
    for i in x:
        cent.append(X_train[i])
    cent = np.array(cent)
    return cent

def euclid(cent,X_train):
    cluster = []
    for i in range(0,len(cent)):
        sub = np.add(X_train,-cent[i])
        sub = np.square(sub)
        sub_sum = np.sum(sub,axis=1)
        sub_sum = np.sqrt(sub_sum)
        cluster.append(sub_sum)
    return np.array(cluster)

def clp_l(cluster):
    cluster = np.argmin(cluster,axis=0)
    clp = []
    for i in range (0,len(cent)):
        cols=[]
        for j in range(0,len(cluster)):
            if cluster[j]==i:
                cols.append(j)
        clp.append(cols)
    return clp

def upcent(clp,X_train):
    cent = []
    
    for x in range(0,len(clp)):
        X_c = []
        for y in range(0,len(clp[x])):
            X_c.append(X_train[clp[x][y]])
        X_c = np.array(X_c)
        sum_X = np.sum(X_c,axis=0)
        sum_X= sum_X/len(X_c)
        cent.append(sum_X)
            
    cent = np.array(cent)
    return cent
def fit(cent,new_cent,X_train):
    while np.array_equal(cent, new_cent)!=True:
        cent = new_cent
        cluster = euclid(cent,X_train)
        clp = clp_l(cluster)
        new_cent = upcent(clp,X_train)
    return new_cent,clp

def t_label(clp):
    y_majority = []
    for i in range(0,len(clp)):
        col=[]
        for y in range(0,len(clp[i])):
            col.append(int(Y_train[clp[i][y]]))
        y_majority.append(col)
    return y_majority

def c_label(y_majority):
    y_label = []
    label = []
    for i in range(0,len(y_majority)):
    
        x = np.array(y_majority[i])
        unique, counts = np.unique(x, return_counts=True)
        y_label.append(np.asarray((unique, counts)).T)
    y_label = np.array(y_label)
    return y_label
def classify_m(y_label):
    cl = []
    for i in range(0,len(y_label)):
        i,j = np.unravel_index(y_label[i].argmax(), y_label[i].shape)
        cl.append(i)
    return cl
def predict(cent,X_test):
    cluster = euclid(cent,X_test)
    clp = clp_l(cluster)
    return clp
def lablled_predict(y_predict,Y_test):
    y_actual = []
    y_predict_l = []
    for i in range(0,len(y_predict)):
        cols = []
        col =[]
        for j in range(0,len(y_predict[i])):
            cols.append(cl[i])
            col.append(Y_test[y_predict[i][j]])
        y_actual.append(col)
        y_predict_l.append(cols)
    return y_predict_l,y_actual
def accuracy(y_predict_l , y_actual):
    wc = []
    for i in range(0,len(y_predict_l)):
        c=0
        for j in range(0,len(y_predict_l[i])):
            if y_predict_l[i][j]==y_actual[i][j]:
                c=c+1
        if len(y_predict_l[i]) ==0:
            c = 100
        else :    
            c = (c/len(y_predict_l[i]))*100
        wc.append(c)
    return wc
        
def w_accuracy(y_predict_l,Y_test,y_actual,c):
    wc = []
    h=0
    for i in range(0,len(y_predict_l)):
        v = (c[i])* (len(y_predict_l[i])/(len(Y_test)))
        wc.append(v)
        h = h+v
    return h , wc
            
    
    
    

# K-MEANS FOR 3

In [7]:
cent = intialize_centroid(3)
cluster = euclid(cent,X_train)
clp = clp_l(cluster)
new_cent = upcent(clp,X_train)
new_cent,clp = fit(cent, new_cent, X_train)
y_majority = t_label(clp)
y_label = c_label(y_majority)
cl = classify_m(y_label)
y_pred = predict(new_cent,X_test)
y_predict_l,y_actual = lablled_predict(y_pred,Y_test)
c =accuracy(y_predict_l,y_actual)
h,wc = w_accuracy(y_predict_l,Y_test,y_actual,c)

In [8]:
print("Predicitions")
print(y_majority)
print("All the type of classification in every cluster")
print(y_label)
print("Maximum classified class for each cluster")
print(cl)
print("Actual Output")
print(y_actual)
print("Predicted Output")
print(y_predict_l)
print("Normal Accuracy")
print(c)
print("Weighted Accuracies")
print(wc)
print("Total Weighted Accuracy")
print(h)

Predicitions
[[0, 2, 1, 2, 1, 2, 0, 0, 0, 1, 1, 0, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 2, 0, 0, 0, 0, 1, 2, 1, 0, 0, 2, 2, 1, 2, 2, 0, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 0, 2, 2, 0, 1, 2, 1, 1, 1, 2, 0, 1, 0, 1, 2, 2, 0, 1, 2, 2, 0, 1, 1, 0, 1, 2, 2, 2, 0, 1, 0, 2, 1, 0, 1, 1, 1, 1, 1, 1, 1]]
All the type of classification in every cluster
[[[ 0  5]
  [ 1 15]
  [ 2  5]]

 [[ 0  9]
  [ 1 18]
  [ 2  7]]

 [[ 0 10]
  [ 1 21]
  [ 2 12]]]
Maximum classified class for each cluster
[1, 1, 1]
Actual Output
[[2, 1, 0], [1, 1, 2], [2, 0, 2, 1, 1, 1, 0, 0, 0, 0, 2, 2]]
Predicted Output
[[1, 1, 1], [1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]
Normal Accuracy
[33.33333333333333, 66.66666666666666, 25.0]
Weighted Accuracies
[5.5555555555555545, 11.111111111111109, 16.666666666666664]
Total Weighted Accuracy
33.33333333333333


# K-MEANS FOR 6

In [9]:
cent = intialize_centroid(6)
cluster = euclid(cent,X_train)
clp = clp_l(cluster)
new_cent = upcent(clp,X_train)
new_cent,clp = fit(cent, new_cent, X_train)
y_majority = t_label(clp)
y_label = c_label(y_majority)
cl = classify_m(y_label)
y_pred = predict(new_cent,X_test)
y_predict_l,y_actual = lablled_predict(y_pred,Y_test)
c =accuracy(y_predict_l,y_actual)
h,wc = w_accuracy(y_predict_l,Y_test,y_actual,c)

In [17]:
print("Predicitions")
print(y_majority)
print("All the type of classification in every cluster")
print(y_label)
print("Maximum classified class for each cluster")
print(cl)
print("Actual Output")
print(y_actual)
print("Predicted Output")
print(y_predict_l)
print("Normal Accuracy")
print(c)
print("Weighted Accuracies")
print(wc)
print("Total Weighted Accuracy")
print(h)

Predicitions
[[2, 0, 1, 0, 1, 2, 2, 1, 2, 1, 0, 2, 1, 1, 1, 1, 1], [0, 1, 0, 1, 1, 1, 1], [0, 1, 0, 0, 1, 2, 1, 1, 1, 1, 1, 1, 1], [0, 0, 2, 0, 0, 0, 1, 2, 0, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1], [0, 2, 1, 2, 2, 0, 0, 1, 0, 2, 2, 1, 1, 1, 1], [1, 1, 0, 2, 1, 2, 1, 1, 2, 1, 1, 2, 1, 2, 0, 1, 1, 0, 0, 2, 0, 2, 1, 0, 1, 1, 1, 1, 1]]
All the type of classification in every cluster
[array([[0, 3],
        [1, 9],
        [2, 5]]) array([[0, 2],
                        [1, 5]]) array([[0, 3],
                                        [1, 9],
                                        [2, 1]]) array([[0, 6],
                                                        [1, 9],
                                                        [2, 6]])
 array([[0, 4],
        [1, 6],
        [2, 5]]) array([[ 0,  6],
                        [ 1, 16],
                        [ 2,  7]])]
Maximum classified class for each cluster
[1, 1, 1, 1, 1, 1]
Actual Output
[[2, 0, 2, 1, 0, 0, 0, 2], [2], [2, 1, 0, 0], [1, 1], []

# K-MEANS FOR 9

In [15]:
cent = intialize_centroid(6)
cluster = euclid(cent,X_train)
clp = clp_l(cluster)
new_cent = upcent(clp,X_train)
new_cent,clp = fit(cent, new_cent, X_train)
y_majority = t_label(clp)
y_label = c_label(y_majority)
cl = classify_m(y_label)
y_pred = predict(new_cent,X_test)
y_predict_l,y_actual = lablled_predict(y_pred,Y_test)
c =accuracy(y_predict_l,y_actual)
h,wc = w_accuracy(y_predict_l,Y_test,y_actual,c)

In [16]:
print("Predicitions")
print(y_majority)
print("All the type of classification in every cluster")
print(y_label)
print("Maximum classified class for each cluster")
print(cl)
print("Actual Output")
print(y_actual)
print("Predicted Output")
print(y_predict_l)
print("Normal Accuracy")
print(c)
print("Weighted Accuracies")
print(wc)
print("Total Weighted Accuracy")
print(h)

Predicitions
[[2, 0, 1, 0, 1, 2, 2, 1, 2, 1, 0, 2, 1, 1, 1, 1, 1], [0, 1, 0, 1, 1, 1, 1], [0, 1, 0, 0, 1, 2, 1, 1, 1, 1, 1, 1, 1], [0, 0, 2, 0, 0, 0, 1, 2, 0, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1], [0, 2, 1, 2, 2, 0, 0, 1, 0, 2, 2, 1, 1, 1, 1], [1, 1, 0, 2, 1, 2, 1, 1, 2, 1, 1, 2, 1, 2, 0, 1, 1, 0, 0, 2, 0, 2, 1, 0, 1, 1, 1, 1, 1]]
All the type of classification in every cluster
[array([[0, 3],
        [1, 9],
        [2, 5]]) array([[0, 2],
                        [1, 5]]) array([[0, 3],
                                        [1, 9],
                                        [2, 1]]) array([[0, 6],
                                                        [1, 9],
                                                        [2, 6]])
 array([[0, 4],
        [1, 6],
        [2, 5]]) array([[ 0,  6],
                        [ 1, 16],
                        [ 2,  7]])]
Maximum classified class for each cluster
[1, 1, 1, 1, 1, 1]
Actual Output
[[2, 0, 2, 1, 0, 0, 0, 2], [2], [2, 1, 0, 0], [1, 1], []

# Observation

As our database contains a huge amount of the points classified as plastic i.e we got majority of every classification as plastic hence we have the same classification accuracy for k=3,6,9